In [28]:
import yfinance as yf
import json
import pandas as pd

tesla = yf.Ticker("TSLA")

tesla_info = tesla.info

with open('tesla.json', 'w') as json_file:
    json.dump(tesla_info, json_file, indent=4)

!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/data/tesla.json -O tesla_ref.json

with open('tesla.json') as json_file:
    tesla_info_ref = json.load(json_file)

print("Tesla company information preview:\n")
for key, value in list(tesla_info_ref.items())[:10]:
    print(f"{key}: {value}")

tesla_share_price_data = tesla.history(period="max")

print("\nTesla Stock Price Data (head):")
tesla_share_price_data.head()


zsh:1: command not found: wget
Tesla company information preview:

address1: 1 Tesla Road
city: Austin
state: TX
zip: 78725
country: United States
phone: 512 516 8177
website: https://www.tesla.com
industry: Auto Manufacturers
industryKey: auto-manufacturers
industryDisp: Auto Manufacturers

Tesla Stock Price Data (head):


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


In [43]:
import requests
from bs4 import BeautifulSoup
import warnings
import brotli

warnings.filterwarnings("ignore", category=FutureWarning)

url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.google.com/",
    "Connection": "keep-alive",
}

response = requests.get(url, headers=headers)

try:
    if response.headers.get("Content-Encoding") == "br":
        data = brotli.decompress(response.content).decode("utf-8")
    elif response.headers.get("Content-Encoding") == "gzip":
        import gzip
        data = gzip.decompress(response.content).decode("utf-8")
    else:
        data = response.text
except Exception as e:
    print("Error decoding response:", e)
    data = response.text

if "Cloudflare" in data or "Attention Required" in data:
    raise ConnectionError("Blocked by Cloudflare. Try VPN or re-run in a few minutes.")

soup = BeautifulSoup(data, "html.parser")
tables = soup.find_all("table", class_="historical_data_table table")

if len(tables) < 2:
    raise ValueError("Quarterly revenue table not found.")

quarterly_table = tables[1]

header_text = quarterly_table.find("th").text.strip()
print(f"Table Title: {header_text}\n")

tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

for row in quarterly_table.find("tbody").find_all("tr"):
    cols = row.find_all("td")
    if len(cols) == 2:
        date = cols[0].text.strip()
        revenue = cols[1].text.strip()
        if date and revenue and revenue != "—":
            tesla_revenue = pd.concat(
                [tesla_revenue,
                 pd.DataFrame({"Date": [date], "Revenue": [revenue]})],
                ignore_index=True
            )

tesla_revenue["Revenue"] = tesla_revenue["Revenue"].str.replace("$", "").str.replace(",", "")
tesla_revenue = tesla_revenue[tesla_revenue["Revenue"] != ""]
tesla_revenue.dropna(inplace=True)

display(tesla_revenue.tail())


Table Title: Tesla Quarterly Revenue(Millions of US $)



,Date,Revenue
55,2011-12-31,39
56,2011-09-30,58
57,2011-06-30,58
58,2011-03-31,49
59,2010-12-31,36


In [45]:
gme = yf.Ticker("GME")

gme_info = gme.info
with open('gme.json', 'w') as json_file:
    json.dump(gme_info, json_file, indent=4)

gme_data = gme.history(period="max")

gme_data.reset_index(inplace=True)
print("GameStop Stock Price Data (head):")
display(gme_data.head())

GameStop Stock Price Data (head):


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716073,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615921,1.662210,1.603296,1.662210,6892800,0.0,0.0


In [47]:
url = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.google.com/",
    "Connection": "keep-alive",
}

response = requests.get(url, headers=headers)


try:
    if response.headers.get("Content-Encoding") == "br":
        data = brotli.decompress(response.content).decode("utf-8")
    elif response.headers.get("Content-Encoding") == "gzip":
        import gzip
        data = gzip.decompress(response.content).decode("utf-8")
    else:
        data = response.text
except Exception as e:
    print("Error decoding response:", e)
    data = response.text

soup = BeautifulSoup(data, "html.parser")

tables = soup.find_all("table", class_="historical_data_table table")

if len(tables) < 2:
    raise ValueError("Could not find GameStop quarterly revenue table.")

quarterly_table = tables[1]

gme_revenue = pd.DataFrame(columns=["Date", "Revenue"])

for row in quarterly_table.find("tbody").find_all("tr"):
    cols = row.find_all("td")
    if len(cols) == 2:
        date = cols[0].text.strip()
        revenue = cols[1].text.strip()
        if date and revenue and revenue != "—":
            gme_revenue = pd.concat(
                [gme_revenue,
                 pd.DataFrame({"Date": [date], "Revenue": [revenue]})],
                ignore_index=True
            )

gme_revenue["Revenue"] = gme_revenue["Revenue"].str.replace("$", "").str.replace(",", "")
gme_revenue = gme_revenue[gme_revenue["Revenue"] != ""]
gme_revenue.dropna(inplace=True)

print("GameStop Quarterly Revenue Data Extracted Successfully!\n")

display(gme_revenue.tail())

GameStop Quarterly Revenue Data Extracted Successfully!



,Date,Revenue
54,2012-01-31,3579
55,2011-10-31,1947
56,2011-07-31,1744
57,2011-04-30,2281
58,2011-01-31,3693


In [18]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Make sure Plotly renders in your notebook
pio.renderers.default = "notebook_connected"

tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max").reset_index()
print("✅ Tesla Stock Data fetched:", len(tesla_data), "rows")

url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/128.0.0.0 Safari/537.36"
    ),
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
}

response = requests.get(url, headers=headers, timeout=30)
soup = BeautifulSoup(response.text, "html.parser")

# Find the correct table (Quarterly Revenue)
target_table = None
for table in soup.find_all("table", class_="historical_data_table"):
    header = table.find("th")
    if header and "Quarterly Revenue" in header.text:
        target_table = table
        break

if target_table is None:
    raise RuntimeError("⚠️ Could not find Tesla Quarterly Revenue table.")

# Extract table data
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])
for row in target_table.find("tbody").find_all("tr"):
    cols = row.find_all("td")
    if len(cols) == 2:
        date = cols[0].text.strip()
        revenue = cols[1].text.strip().replace("$", "").replace(",", "")
        if date and revenue and revenue not in ["—", ""]:
            tesla_revenue = pd.concat(
                [tesla_revenue, pd.DataFrame([[date, revenue]], columns=["Date", "Revenue"])],
                ignore_index=True
            )

# Clean data
tesla_revenue["Date"] = pd.to_datetime(tesla_revenue["Date"])
tesla_revenue["Revenue"] = tesla_revenue["Revenue"].astype(float)

fig = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.1,
    subplot_titles=("Historical Share Price", "Historical Revenue")
)

# Plot Stock Price
fig.add_trace(
    go.Scatter(
        x=tesla_data["Date"],
        y=tesla_data["Close"],
        mode="lines",
        name="Tesla Stock Price",
        line=dict(color="royalblue")
    ),
    row=1, col=1
)

# Plot Revenue
fig.add_trace(
    go.Scatter(
        x=tesla_revenue["Date"],
        y=tesla_revenue["Revenue"],
        mode="lines",
        name="Tesla Quarterly Revenue",
        line=dict(color="firebrick")
    ),
    row=2, col=1
)

# Layout and labels
fig.update_layout(
    title_text="Tesla",
    showlegend=False,
    height=800,
    width=1000,
    template="plotly_white"
)
fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
fig.update_yaxes(title_text="Revenue (US$ Millions)", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=1)

fig.show()


✅ Tesla Stock Data fetched: 3863 rows
✅ Tesla Revenue Data fetched: 60 quarters


In [17]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

gme = yf.Ticker("GME")
gme_data = gme.history(period="max").reset_index()

url = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/128.0.0.0 Safari/537.36"
    ),
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.google.com/",
}
response = requests.get(url, headers=headers, timeout=30)
soup = BeautifulSoup(response.text, "html.parser")

# Find the table
target_table = None
for table in soup.find_all("table", class_="historical_data_table"):
    header = table.find("th")
    if header and "Quarterly Revenue" in header.text:
        target_table = table
        break

if target_table is None:
    raise RuntimeError("⚠️ Could not find GameStop Quarterly Revenue table.")

# Parse table data
gme_revenue = pd.DataFrame(columns=["Date", "Revenue"])
for row in target_table.find("tbody").find_all("tr"):
    cols = row.find_all("td")
    if len(cols) == 2:
        date = cols[0].text.strip()
        revenue = cols[1].text.strip().replace("$", "").replace(",", "")
        if date and revenue and revenue not in ["—", ""]:
            gme_revenue = pd.concat(
                [gme_revenue, pd.DataFrame([[date, revenue]], columns=["Date", "Revenue"])],
                ignore_index=True
            )

# Clean data
gme_revenue["Date"] = pd.to_datetime(gme_revenue["Date"])
gme_revenue["Revenue"] = gme_revenue["Revenue"].astype(float)

fig = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.1,
    subplot_titles=("Historical Share Price", "Historical Revenue")
)

# Stock Price subplot (blue)
fig.add_trace(
    go.Scatter(
        x=gme_data["Date"],
        y=gme_data["Close"],
        mode="lines",
        name="GameStop Stock Price",
        line=dict(color="royalblue")
    ),
    row=1, col=1
)

# Revenue subplot (red)
fig.add_trace(
    go.Scatter(
        x=gme_revenue["Date"],
        y=gme_revenue["Revenue"],
        mode="lines",
        name="GameStop Quarterly Revenue",
        line=dict(color="firebrick")
    ),
    row=2, col=1
)

# Layout styling
fig.update_layout(
    title_text="GameStop",
    showlegend=False,
    height=800,
    width=1000,
    template="plotly_white",
)

# Axis labels
fig.update_yaxes(title_text="Price (USD)", row=1, col=1)
fig.update_yaxes(title_text="Revenue (US$ Millions)", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=1)


pio.renderers.default = "notebook_connected"

fig.show()


✅ GameStop Stock Data fetched: 5971 rows
✅ GameStop Revenue Data fetched: 59 quarters
